In [9]:
import cv2
import os
from pathlib import Path
from decord import VideoReader
from decord import cpu, gpu

# 3 Second Clips

# Label 0 and 1 

In [45]:

input_video = "Videos/7941D100.mp4"
output_dir = "Clips"
annotation_file = "Single_annotation.txt"

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Open the video file and get the properties
video = cv2.VideoCapture(input_video)
fps = video.get(cv2.CAP_PROP_FPS)
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# Read the seizure start and end times from the annotation file
seizure_times = []
with open("video_annot.txt", "r") as seizure_file:
    for line in seizure_file:
        values = eval(line.strip())
        if len(values) == 4:
            video_path, video_id, patient_id, time_range = values
            if video_path == input_video:
                start_time, end_time = time_range
                seizure_times.append((start_time, end_time))  # Keep the times in seconds

# Convert seizure times from seconds to frames
seizure_frames = []
for start, end in seizure_times:
    start_frame = int(start * fps)
    end_frame = int(end * fps)
    seizure_frames.append((start_frame, end_frame))

# Calculate the number of 3-second segments
clip_duration = 3  # Duration of each clip in seconds
clip_frames = int(clip_duration * fps)
num_segments = (total_frames + clip_frames - 1) // clip_frames

print("Number of segments is:", num_segments)

# Initialize the segment counter
segment_count = 0

# Open the annotation file for writing
with open(annotation_file, "w") as f:
    # Iterate over the segments
    for i in range(num_segments):
        # Set the starting frame for the current segment
        start_frame = i * clip_frames
        end_frame = min(start_frame + clip_frames, total_frames)

        # Set the video position to the starting frame
        video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

        # Initialize the frames list for the current segment
        segment_frames = []

        # Read frames for the current segment
        for _ in range(end_frame - start_frame):
            ret, frame = video.read()
            if ret:
                segment_frames.append(frame)
            else:
                break

        # If frames were successfully read for the segment
        if len(segment_frames) > 0:
            # Create the output segment path
            segment_name = f"{patient_id}_{video_id}_Seg_{segment_count}.mp4"
            segment_path = os.path.join(output_dir, segment_name)

            # Write the segment frames to the output video file
            out = cv2.VideoWriter(segment_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (frame.shape[1], frame.shape[0]))
            for frame in segment_frames:
                out.write(frame)
            out.release()

            # Check if the current segment overlaps with any seizure time
            is_seizure = any(seizure_start <= start_frame < seizure_end or
                                seizure_start < end_frame <= seizure_end
                              for seizure_start, seizure_end in seizure_frames)

            # Write the annotation for the current segment
            annotation = f"{segment_path.replace('\\', '/')} {1 if is_seizure else 0}\n"
            f.write(annotation)

            # Increment the segment counter
            segment_count += 1

# Release the video capture
video.release()
print("Video segmentation and annotation completed.")

Number of segments is: 109
Video segmentation and annotation completed.


In [22]:
print("Total frames : " , total_frames)


Total frames :  9635


In [38]:
print(seizure_times)

[(136, 170)]
